In [1]:
%matplotlib inline

In [2]:
from gahyparopt.widgets import *
from gahyparopt.GAUtilities import *

In [3]:
mnist = load_data()

In [4]:
number_of_players = 10

In [5]:
players = ["Player{}".format(i) for i in range(number_of_players)]

In [6]:
players

['Player0',
 'Player1',
 'Player2',
 'Player3',
 'Player4',
 'Player5',
 'Player6',
 'Player7',
 'Player8',
 'Player9']

In [7]:
games = dict([(p, game_instance(
            ga=GADriver(
            layer_counts=HIDDEN_LAYER_COUNT,
            no_neurons=HIDDEN_LAYER_NEURONS,
            rates=HIDDEN_LAYER_RATE,
            activations=HIDDEN_LAYER_ACTIVATIONS,
            layer_types=HIDDEN_LAYER_TYPE,
            optimizers=MODEL_OPTIMIZER,
            population_size=1,
            best_candidates_count=BEST_CANDIDATES_COUNT,
            random_candidates_count=RANDOM_CANDIDATES_COUNT,
            optimizer_mutation_probability=OPTIMIZER_MUTATION_PROBABILITY,
            layer_mutation_probability=HIDDEN_LAYER_MUTATION_PROBABILITY,
        ))
    ) for p in players])

In [8]:
%%capture
# Init all players with data and model.
for player, game in games.items():
    game.name = player
    game.data = mnist
    game.individuum = create_start_individuum(game.ga)[0]
    
    
    
    
    

2022-05-03 22:08:48.947498: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-05-03 22:08:49.019098: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:1a:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-05-03 22:08:49.019621: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 1 with properties: 
pciBusID: 0000:1b:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-05-03 22:08:49.020164: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 2 with properties: 
pciBusID: 0000:3d:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBan

In [9]:
games['Player0'].individuum.ml_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            (None, 784)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                9

In [11]:
generation = 0

In [17]:
# Training
clear_keras_session()
for player, game in games.items():
    # sync_remote_to_local()
    if generation > 0:
        games[player].individuum = read_chromosome(game.name)
    
    try:
        games[player].history = train_individuum(game.ga, game.data, game.individuum)
        
    except:
        games[player].history = None
    
    if games[player].history is not None:
        try:
                loss, accuracy = evaluate_model(game.individuum.ml_model, game.data['x_val'], y=game.data['y_val'])
        except:
            accuracy = 0.0
            loss = 100.0

    else:
        accuracy = 0.0
        loss = 100.0

    games[player].individuum.loss = loss
    games[player].individuum.accuracy = accuracy
    
    write_chromosome(game.name, game.individuum)
    # sync_local_to_remote(game.name)
    
generation += 1 
    
    

Epoch 1/10
10/10 [==============================] - 2s 150ms/step - loss: 1.9747 - accuracy: 0.3668 - val_loss: 1.4356 - val_accuracy: 0.6672
Epoch 2/10
10/10 [==============================] - 1s 142ms/step - loss: 1.2146 - accuracy: 0.7227 - val_loss: 0.9498 - val_accuracy: 0.7997
Epoch 3/10
10/10 [==============================] - 1s 141ms/step - loss: 0.8680 - accuracy: 0.8023 - val_loss: 0.7231 - val_accuracy: 0.8351
Epoch 4/10
10/10 [==============================] - 1s 140ms/step - loss: 0.6957 - accuracy: 0.8323 - val_loss: 0.5988 - val_accuracy: 0.8567
Epoch 5/10
10/10 [==============================] - 1s 139ms/step - loss: 0.5915 - accuracy: 0.8547 - val_loss: 0.5235 - val_accuracy: 0.8694
Epoch 6/10
10/10 [==============================] - 1s 136ms/step - loss: 0.5269 - accuracy: 0.8664 - val_loss: 0.4722 - val_accuracy: 0.8810
Epoch 7/10
10/10 [==============================] - 1s 141ms/step - loss: 0.4809 - accuracy: 0.8751 - val_loss: 0.4361 - val_accuracy: 0.8891
Epoch 

# Danger zone!!!

In [17]:
delete_local()
delete_remote()